In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from dask.distributed import Client

In [4]:
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')

In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/crime.csv',encoding="ISO-8859–1")

In [11]:
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [126]:
%%time
def create_exprs(columns, funcs):
    exprs = []
    for f in funcs:
        exprs.append(getattr(df[columns], f)())
    return exprs

agg_func=["min","max","mean"]
a=create_exprs(["INCIDENT_NUMBER", "OFFENSE_CODE"], agg_func)

# dd.compute(df.min(), df.INCIDENT_NUMBER.max(), df.mean())

result =dd.compute(a)

Wall time: 1.48 s


In [133]:
r= {}
for i, agg in enumerate(result[0]):
    agg_name = agg_func[i]
    r[agg_name]={}
    for col, value in agg.items():        
        r[agg_name][col]=value

print(r)

{'min': {'INCIDENT_NUMBER': 142052550.0, 'OFFENSE_CODE': 111.0}, 'max': {'INCIDENT_NUMBER': 'I182070945', 'OFFENSE_CODE': 3831}, 'mean': {'OFFENSE_CODE': 2317.546956339145}}


In [56]:
# Another approach
# from dask.array import stats
# x = da.random.beta(1, 1, size=(1000,), chunks=10)
# k, s, m = [stats.kurtosis(x), stats.skew(x), stats.moment(x, 5)]
# dask.compute(k, s, m)

(1.7612340817172787, -0.064073498030693302, -0.00054523780628304799)

(1.7612340817172787, -0.0640734980306933, -0.000545237806283048)

In [4]:
from optimus import optimus as Optimus

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 
`formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly


In [57]:
data = [['tom', 10], ['nick', 15], ['juli', 0],['argenis', 10]]

# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['Name', 'Age']) 
df.cols.count_by_dtypes("*")

{'Name': {'int': 0,
  'decimal': 0,
  'string': 0,
  'date': 0,
  'boolean': 0,
  'binary': 0,
  'array': 0,
  'object': 4,
  'null': 0,
  'missing': 0},
 'Age': {'int': 0,
  'decimal': 0,
  'string': 0,
  'date': 0,
  'boolean': 0,
  'binary': 0,
  'array': 0,
  'object': 0,
  'null': 0,
  'missing': 0}}

In [58]:
df

,Name,Age
0,tom,10
1,nick,15
2,juli,0
3,argenis,10


In [65]:
# df[df == 0].count(axis=0)
# Speedy https://stackoverflow.com/questions/35277075/python-pandas-counting-the-occurrences-of-a-specific-value
def zeros(series):
    return (series.values==0).sum()

zeros(df["Age"])

1

In [135]:
def percentile_agg(col_name, values):  
    return col_name.quantile(values).to_dict()
    

percentile_agg(df["Age"], [0.1, 0.5])

{0.1: 3.0000000000000004, 0.5: 10.0}

In [136]:
df.agg(
    {
#         # find the min, max, and sum of the duration column
#         'Name': [min, max, sum],
#          # find the number of network type entries
#         'Age': "count",
        # min, first, and number of unique dates per group
        'Age': [min, 'first', 'nunique', percentile_agg("Age",[0.1, 0.5])]
    }
).to_dict()

AttributeError: 'str' object has no attribute 'quantile'

In [37]:
print(df)

   Name Age
0   tom  10
1  nick  15
2  juli   0


In [138]:
from optimus.profiler.profiler import Profiler
p = Profiler()
print(df.functions)
p.columns_agg(df)

<optimus.pandas.functions.functions.<locals>.Functions object at 0x000001F961E9E438>
<optimus.pandas.functions.functions.<locals>.Functions object at 0x000001F961E7EF60>
[(<function functions.<locals>.Functions.count_uniques_agg at 0x000001F961EED1E0>, ('Name', True)), (<function functions.<locals>.Functions.count_uniques_agg at 0x000001F961EED1E0>, ('Age', True))]


AttributeError: 'str' object has no attribute 'value_counts'